لتشغيل دفاتر الملاحظات التالية، إذا لم تكن قد فعلت ذلك بعد، تحتاج إلى نشر نموذج يستخدم `text-embedding-ada-002` كنموذج أساسي وتعيين اسم النشر داخل ملف .env كـ `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

بعد ذلك، سنقوم بتحميل مؤشر التضمين في إطار بيانات Pandas. يتم تخزين مؤشر التضمين في ملف JSON يسمى `embedding_index_3m.json`. يحتوي مؤشر التضمين على التضمينات لكل من نصوص YouTube حتى أواخر أكتوبر 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

بعد ذلك، سنقوم بإنشاء دالة باسم `get_videos` تبحث في فهرس التضمين عن الاستعلام المطلوب. ستعيد الدالة أفضل 5 فيديوهات الأكثر تشابهًا مع الاستعلام. تعمل الدالة بالطريقة التالية:

1. أولاً، يتم إنشاء نسخة من فهرس التضمين.
2. بعد ذلك، يتم حساب تضمين الاستعلام باستخدام واجهة برمجة تطبيقات تضمين OpenAI.
3. ثم يتم إنشاء عمود جديد في فهرس التضمين باسم `similarity`. يحتوي هذا العمود على التشابه الكوني بين تضمين الاستعلام وتضمين كل مقطع فيديو.
4. بعد ذلك، يتم تصفية فهرس التضمين حسب عمود `similarity`. يتم الاحتفاظ فقط بالفيديوهات التي لديها تشابه كوني أكبر من أو يساوي 0.75.
5. أخيرًا، يتم ترتيب فهرس التضمين حسب عمود `similarity` وتتم إعادة أفضل 5 فيديوهات.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

هذه الدالة بسيطة جدًا، فهي فقط تطبع نتائج استعلام البحث.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. أولاً، يتم تحميل فهرس التضمين داخل إطار بيانات Pandas.
2. بعد ذلك، يُطلب من المستخدم إدخال استعلام.
3. ثم يتم استدعاء دالة `get_videos` للبحث في فهرس التضمين عن الاستعلام.
4. أخيراً، يتم استدعاء دالة `display_results` لعرض النتائج على المستخدم.
5. بعد ذلك، يُطلب من المستخدم إدخال استعلام آخر. تستمر هذه العملية حتى يقوم المستخدم بإدخال `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ar.png)

سيتم مطالبتك بإدخال استعلام. أدخل استعلامك واضغط على زر الإدخال. سيعرض التطبيق قائمة بالفيديوهات المتعلقة بالاستعلام الذي أدخلته. كما سيعرض التطبيق رابطاً للمكان في الفيديو الذي توجد فيه إجابة السؤال.

إليك بعض الاستعلامات التي يمكنك تجربتها:

- ما هو Azure Machine Learning؟
- كيف تعمل الشبكات العصبية الالتفافية؟
- ما هي الشبكة العصبية؟
- هل يمكنني استخدام Jupyter Notebooks مع Azure Machine Learning؟
- ما هو ONNX؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
